### import graph from wikidata sparql into neo4j

In [1]:
import py2neo as pn
import pandas as pd

#### connect neo4j

In [2]:
graph = pn.Graph("bolt://localhost:7687",password="monk_neo4j")

In [18]:
# nodes = graph.run("MERGE (o:Object {name: 'actor_1'}) RETURN o")
# nodes = graph.run("MERGE (o:wd_node {title: 'wd_title_1'}) RETURN o")
# nodes = graph.run("MATCH (o:Object {name: 'actor_1'}) DELETE o") #delete test node
# nodes = graph.run("MATCH (o:wd_node {title: 'wd_title_1'}) DELETE o") #delete test node

o
(_1:wd_node {title: 'wd_title_1'})


#### connect to wikidata sparql API

In [5]:
# https://people.wikimedia.org/~bearloga/notes/wdqs-python.html
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [7]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [56]:
def query_sp_raw(sparql, query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = results['results']['bindings']
    return results_df

In [65]:
def query_graph(entity_id):
    # graph with all subclasses of entity_id
    q = f"""
    SELECT ?item ?itemLabel ?linkTo {{
        wd:{entity_id} wdt:P279* ?item
        OPTIONAL {{ ?item wdt:P279 ?linkTo }}
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language 'en' }}
    }}
    """
    # print(q)
    return query_sp_raw(sparql, q)


In [68]:
# d1 = query_graph('Q5')
# d1

30

In [9]:
q = """
PREFIX gas: <http://www.bigdata.com/rdf/gas#>

SELECT ?item ?itemLabel ?linkTo {
SERVICE gas:service {
    gas:program gas:gasClass "com.bigdata.rdf.graph.analytics.SSSP" ;
                gas:in wd:Q5 ;
                gas:traversalDirection "Forward" ;
                gas:out ?item ;
                gas:out1 ?depth ;
                gas:maxIterations 10 ;
                gas:linkType wdt:P279 .
  }
OPTIONAL { ?item wdt:P279 ?linkTo }
SERVICE wikibase:label {bd:serviceParam wikibase:language "en" }
} 
        """
res = query_sp_raw(sparql, q)


In [52]:
len(res)

30

In [22]:
# res[0]['item']['value']
# res[0]['itemLabel']['value']
# res[0]['linkTo']['value']
# nodes = graph.run("MERGE (o:wd_node {title: 'wd_title_1'}) RETURN o")

In [35]:
def uri_id(uri):
    return uri.split('/')[-1]


" \n        MERGE (o:wd_node {title: 'jora'}) RETURN o\n        \n    "

In [87]:
subject_title = 'tit'
subject_id = 1
entity_id = 2
neo4j_merge = """ 
            MERGE (o:wd_node {{id: '{subject_id}', title: '{subject_title}'}})
            SET o:{entity_id}{entity}
            RETURN o
        """.format(
            subject_id=subject_id, 
            subject_title=subject_title,
            entity_id=entity_id, 
            entity=':term' if subject_id == entity_id else '')#
neo4j_merge

" \n            MERGE (o:wd_node {id: '1', title: 'tit'})\n            SET o:2\n            RETURN o\n        "

In [89]:
def create_graph(res, entity_id):
    # entity_id - id of search term (entity)
    relation_type = 'subclass_of'# todo: make an argument
    for i in res:# create nodes
        subject_id = uri_id(i['item']['value'])
        subject_title = i['itemLabel']['value']
        try:
            object_id = uri_id(i['linkTo']['value'])
        except Exception as e:
            print(f'[error]: {e}')
            print(f"in subject: {subject_title}({subject_id})")
            object_id = None
        neo4j_merge = """ 
            MERGE (o:wd_node {{id: '{subject_id}', title: '{subject_title}'}})
            SET o:{entity_id}{entity}
            RETURN o
        """.format(
            subject_id=subject_id, 
            subject_title=subject_title,
            entity_id=entity_id, 
            entity=':search_term' if subject_id == entity_id else '')# info: add entity_id as node-label for furter exploration relations between entities
        # print(neo4j_merge)
        graph.run(neo4j_merge)
        # print(f"subject: {subject_title}({subject_id}) was added")
    for i in res:# add relations between nodes
        subject_id = uri_id(i['item']['value'])
        subject_title = i['itemLabel']['value']
        try:
            object_id = uri_id(i['linkTo']['value'])
        except Exception as e:
            print(f'[error]: {e}')
            object_id = None
            continue # don't create relation if there is no object
        neo4j_merge = f"""
            MATCH (s:wd_node {{id: '{subject_id}'}})
            MATCH (o:wd_node {{id: '{object_id}'}})
            MERGE (s)-[:{relation_type}]->(o) RETURN s, o
        """
        # print(neo4j_merge)
        graph.run(neo4j_merge)

In [90]:
entity_ids = ['Q5', 'Q3', 'Q47607']
for entity_id in entity_ids:
    create_graph(query_graph(entity_id), entity_id)

[error]: 'linkTo'
in subject: entity(Q35120)
[error]: 'linkTo'
[error]: 'linkTo'
in subject: entity(Q35120)
[error]: 'linkTo'
in subject: predicate(Q4376586)
[error]: 'linkTo'
[error]: 'linkTo'
[error]: 'linkTo'
in subject: entity(Q35120)
[error]: 'linkTo'


#### create nodes

In [49]:
for i in res:
    subject_id = uri_id(i['item']['value'])
    subject_title = i['itemLabel']['value']
    try:
        object_id = uri_id(i['linkTo']['value'])
    except Exception as e:
        print(f'[error]: {e}')
        object_id = None
    relation = 'subclass_of'

    
    neo4j_merge = f""" 
        MERGE (o:wd_node {{id: '{subject_id}', title: '{subject_title}'}}) RETURN o
    """
    # print(neo4j_merge)
    graph.run(neo4j_merge)
    # print(f"subject: {subject_title}({subject_id}) was added")


 
        MERGE (o:wd_node {id: 'Q5', title: 'human'}) RETURN o
    
subject: human(Q5) was added
 
        MERGE (o:wd_node {id: 'Q5', title: 'human'}) RETURN o
    
subject: human(Q5) was added
 
        MERGE (o:wd_node {id: 'Q5', title: 'human'}) RETURN o
    
subject: human(Q5) was added
 
        MERGE (o:wd_node {id: 'Q5', title: 'human'}) RETURN o
    
subject: human(Q5) was added
 
        MERGE (o:wd_node {id: 'Q729', title: 'animal'}) RETURN o
    
subject: animal(Q729) was added
 
        MERGE (o:wd_node {id: 'Q7239', title: 'organism'}) RETURN o
    
subject: organism(Q7239) was added
[error]: 'linkTo'
 
        MERGE (o:wd_node {id: 'Q35120', title: 'entity'}) RETURN o
    
subject: entity(Q35120) was added
 
        MERGE (o:wd_node {id: 'Q72638', title: 'consumer'}) RETURN o
    
subject: consumer(Q72638) was added
 
        MERGE (o:wd_node {id: 'Q72638', title: 'consumer'}) RETURN o
    
subject: consumer(Q72638) was added
 
        MERGE (o:wd_node {id: 'Q154954', t

#### create relations

In [51]:
for i in res:
    subject_id = uri_id(i['item']['value'])
    subject_title = i['itemLabel']['value']
    try:
        object_id = uri_id(i['linkTo']['value'])
    except Exception as e:
        print(f'[error]: {e}')
        object_id = None
        continue # don't create relation if there is no object
    relation = 'subclass_of'

    
    neo4j_merge = f"""
        MATCH (s:wd_node {{id: '{subject_id}'}})
        MATCH (o:wd_node {{id: '{object_id}'}})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    """
    # print(neo4j_merge)
    graph.run(neo4j_merge)


        MATCH (s:wd_node {id: 'Q5'})
        MATCH (o:wd_node {id: 'Q154954'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    

        MATCH (s:wd_node {id: 'Q5'})
        MATCH (o:wd_node {id: 'Q164509'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    

        MATCH (s:wd_node {id: 'Q5'})
        MATCH (o:wd_node {id: 'Q215627'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    

        MATCH (s:wd_node {id: 'Q5'})
        MATCH (o:wd_node {id: 'Q110551885'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    

        MATCH (s:wd_node {id: 'Q729'})
        MATCH (o:wd_node {id: 'Q7239'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    

        MATCH (s:wd_node {id: 'Q7239'})
        MATCH (o:wd_node {id: 'Q27043950'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    
[error]: 'linkTo'

        MATCH (s:wd_node {id: 'Q72638'})
        MATCH (o:wd_node {id: 'Q7239'})
        MERGE (s)-[:subclass_of]->(o) RETURN s, o
    

        MATCH (s:wd_node {